In [6]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm

# Load the CSV file into a DataFrame
y = pd.read_csv('../Original Data/10Targcleaned.csv')



# Display the DataFrame
y.head()

,Datum/Zeit,timestamp_text,PM10 (Stundenmittelwerte),PM2.5 (Stundenmittelwerte),O3 (Stundenmittelwerte),NO2 (Stundenmittelwerte),geo_point_2d
0,2021-11-30 23:00:00+00:00,2021-12-01 00:00:00,7.456,7.023,38.615,24.832,"47.5659354, 7.58192"
1,2021-12-01 00:00:00+00:00,2021-12-01 01:00:00,4.832,2.241,67.784,3.288,"47.5659354, 7.58192"
2,2021-12-01 01:00:00+00:00,2021-12-01 02:00:00,4.682,2.092,70.599,2.459,"47.5659354, 7.58192"
3,2021-12-01 02:00:00+00:00,2021-12-01 03:00:00,5.035,2.398,70.212,2.954,"47.5659354, 7.58192"
4,2021-12-01 03:00:00+00:00,2021-12-01 04:00:00,5.102,2.345,75.184,2.432,"47.5659354, 7.58192"


In [7]:
y_filtered = y[['Datum/Zeit', 'PM10 (Stundenmittelwerte)']]
y_filtered.head()

,Datum/Zeit,PM10 (Stundenmittelwerte)
0,2021-11-30 23:00:00+00:00,7.456
1,2021-12-01 00:00:00+00:00,4.832
2,2021-12-01 01:00:00+00:00,4.682
3,2021-12-01 02:00:00+00:00,5.035
4,2021-12-01 03:00:00+00:00,5.102


In [10]:
# Print the column names to check the exact name of the 'Datum/ Zeit' column
print(y.columns)


Index(['Datum/Zeit', 'timestamp_text', 'PM10 (Stundenmittelwerte)',
       'PM2.5 (Stundenmittelwerte)', 'O3 (Stundenmittelwerte)',
       'NO2 (Stundenmittelwerte)', 'geo_point_2d'],
      dtype='object')


In [12]:
# Print the column names to check for the correct timestamp column name
print(y_filtered.columns)


Index(['Datum/Zeit', 'PM10 (Stundenmittelwerte)'], dtype='object')


In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from joblib import Parallel, delayed

# Assuming 'y_filtered' is your DataFrame
# Ensure the 'PM10 (Stundenmittelwerte)' column is numeric
y_filtered['PM10 (Stundenmittelwerte)'] = pd.to_numeric(y_filtered['PM10 (Stundenmittelwerte)'], errors='coerce')

# Optionally, handle missing values (e.g., by filling or dropping)
y_filtered = y_filtered.dropna(subset=['PM10 (Stundenmittelwerte)'])  # Drop rows with NaN values in the relevant column

# Convert the 'Datum/Zeit' column to datetime format for proper time series handling
y_filtered['Datum/Zeit'] = pd.to_datetime(y_filtered['Datum/Zeit'], errors='coerce')

# Define the length of the dataset and split sizes
n = len(y_filtered)  # Length of the DataFrame y_filtered
train_size_initial = int(n * 0.7)  # 70% of the data for training
test_size = int(n * 0.1)   # 10% of the data for testing
validate_size = int(n * 0.1)  # 10% of the data for validation

# Adjusting the number of combinations to avoid the mismatch
# Generate a manageable number of columns (e.g., based on a subset of combinations)
columns = [f"p={p}_q={q}_P={P}_Q={Q}_s={s}" 
           for p in range(1, 4)  # Limit to 3 values for p
           for q in range(1, 4)  # Limit to 3 values for q
           for P in range(0, 2)  # Limit to 2 values for P
           for Q in range(0, 2)  # Limit to 2 values for Q
           for s in [24]]        # Limit to 1 seasonal period (just 24)

# Now assign the reduced list to the results DataFrame
results_test = pd.DataFrame(np.zeros((1, len(columns))))  # Adjust to match the number of columns
results_test.columns = columns

results_validate = pd.DataFrame(np.zeros((1, len(columns))))  # Adjust to match the number of columns
results_validate.columns = columns

# Use the first 50% of the data for the initial training set (50% of the train_size)
train_size_current = int(train_size_initial * 0.5)  # Initial 50% of the train set
testing_start = train_size_current
testing_end = testing_start + test_size

# Expanding the training set by adding the next 10% in each iteration
training = y_filtered[:train_size_current]  # Initial training set
testing = y_filtered[testing_start:testing_end]  # Testing set
validation_start = testing_end
validation_end = validation_start + validate_size
validation = y_filtered[validation_start:validation_end]  # Validation set

# Function to fit SARIMA model and compute MSE
def fit_sarima(p, q, P, Q, s, training_data, testing, validation):
    # Fit SARIMA model with seasonal order (P, D, Q, s)
    seasonal_order = (P, 1, Q, s)  # D=1 for seasonal differencing
    mod = sm.tsa.ARIMA(training_data['PM10 (Stundenmittelwerte)'], order=(p, 1, q), seasonal_order=seasonal_order)
    res = mod.fit()
    
    # One-step ahead forecast for the testing set
    forecast_test = res.forecast(steps=1, signal_only=False)  # Forecasting just 1 step ahead
    
    # One-step ahead forecast for the validation set
    forecast_validate = res.forecast(steps=1, signal_only=False)  # Forecasting just 1 step ahead
    
    # Calculate Mean Squared Error for the test set
    mse_test = ((testing['PM10 (Stundenmittelwerte)'].values[:1] - forecast_test) ** 2).mean()
    
    # Calculate Mean Squared Error for the validation set
    mse_validate = ((validation['PM10 (Stundenmittelwerte)'].values[:1] - forecast_validate) ** 2).mean()
    
    return mse_test, mse_validate

# Use joblib's Parallel to run the fit_sarima function in parallel for different p, q, P, Q, and s combinations
results = Parallel(n_jobs=-1)(delayed(fit_sarima)(p, q, P, Q, s, y_filtered, testing, validation) 
                              for p in range(1, 4)  # Limit p to 3
                              for q in range(1, 4)  # Limit q to 3
                              for P in range(0, 2)  # Limit P to 2
                              for Q in range(0, 2)  # Limit Q to 2
                              for s in [24])       # Limit to one seasonal period (24)

# Store the results in the DataFrame
for idx, (mse_test, mse_validate) in enumerate(results):
    p_value = idx // 36 + 1  # Calculate p from the index
    q_value = (idx % 36) // 12 + 1  # Calculate q from the index
    P_value = (idx % 12) // 4  # Calculate P from the index
    Q_value = (idx % 4) // 2  # Calculate Q from the index
    s_value = [24][idx % 1]  # Seasonal period (just 24)
    results_test.loc[0, f"p={p_value}_q={q_value}_P={P_value}_Q={Q_value}_s={s_value}"] = mse_test
    results_validate.loc[0, f"p={p_value}_q={q_value}_P={P_value}_Q={Q_value}_s={s_value}"] = mse_validate

# Combine the results into one DataFrame for better display
final_results = pd.concat([results_test, results_validate], axis=0, ignore_index=True)
final_results.index = ["Test MSE", "Validation MSE"]

# Print the results in a more readable format
print(final_results.to_string(index=True))  # Display the results in a tabular format
